# Tutorial 7: Build a Passage Evaluator (Capstone)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-pure-passage-calculus/blob/main/notebooks/tutorial_07_capstone.ipynb)

---

## Building Mund's Machine

In this capstone project, you will build a complete **Passage Evaluator**—a Python interpreter for Mund's Pure Passage Calculus.

Your evaluator will:
1. **Parse** lambda expressions from strings
2. **Evaluate** expressions via beta reduction
3. **Detect** non-termination
4. **Validate** against historical reduction traces

---

## Learning Objectives

By the end of this capstone, you will have:
1. Built a working **parser** for lambda calculus
2. Implemented **beta reduction** with capture-avoiding substitution
3. Created a **REPL** for interactive exploration
4. Validated your implementation against Mund's datasets

## Setup

In [ ]:
import pandas as pd
import re
from dataclasses import dataclass
from typing import Set, Optional, List, Tuple

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions = pd.read_csv(BASE_URL + "passage_calculus_expressions.csv")
reductions = pd.read_csv(BASE_URL + "passage_reductions.csv")
encodings = pd.read_csv(BASE_URL + "church_encodings.csv")

print(f"Loaded {len(expressions)} expressions")
print(f"Loaded {len(reductions)} reduction steps")
print(f"Loaded {len(encodings)} encodings")

## Part 1: Abstract Syntax Tree

First, we define the structure of lambda expressions.

In [ ]:
@dataclass
class Var:
    """A variable: x, y, z, etc."""
    name: str
    
    def __repr__(self):
        return self.name
    
    def __eq__(self, other):
        return isinstance(other, Var) and self.name == other.name
    
    def __hash__(self):
        return hash(("Var", self.name))

@dataclass  
class Lam:
    """A lambda abstraction: λx.body"""
    var: str
    body: 'Expr'
    
    def __repr__(self):
        return f"(λ{self.var}.{self.body})"

@dataclass
class App:
    """An application: (func arg)"""
    func: 'Expr'
    arg: 'Expr'
    
    def __repr__(self):
        return f"({self.func} {self.arg})"

# Type alias
Expr = Var | Lam | App

# Test
identity = Lam('x', Var('x'))
print(f"Identity: {identity}")

K = Lam('x', Lam('y', Var('x')))
print(f"K combinator: {K}")

## Part 2: Free Variables and Substitution

In [ ]:
def free_vars(expr: Expr) -> Set[str]:
    """Return the set of free variables in an expression."""
    if isinstance(expr, Var):
        return {expr.name}
    elif isinstance(expr, Lam):
        return free_vars(expr.body) - {expr.var}
    elif isinstance(expr, App):
        return free_vars(expr.func) | free_vars(expr.arg)
    return set()

# Test
print(f"Free vars in λx.x: {free_vars(Lam('x', Var('x')))}")
print(f"Free vars in λx.y: {free_vars(Lam('x', Var('y')))}")
print(f"Free vars in (x y): {free_vars(App(Var('x'), Var('y')))}")

In [ ]:
# Fresh variable generator
_fresh_counter = 0

def fresh_var(base: str = "v") -> str:
    """Generate a fresh variable name."""
    global _fresh_counter
    _fresh_counter += 1
    return f"{base}{_fresh_counter}"

def reset_fresh():
    """Reset the fresh variable counter."""
    global _fresh_counter
    _fresh_counter = 0

print(fresh_var(), fresh_var(), fresh_var())
reset_fresh()
print(fresh_var())

In [ ]:
def substitute(expr: Expr, var: str, replacement: Expr) -> Expr:
    """
    Substitute replacement for var in expr.
    Uses capture-avoiding substitution.
    """
    if isinstance(expr, Var):
        if expr.name == var:
            return replacement
        return expr
    
    elif isinstance(expr, Lam):
        if expr.var == var:
            # Variable is shadowed
            return expr
        
        # Check for capture
        if expr.var in free_vars(replacement):
            # Need to alpha-convert
            new_var = fresh_var(expr.var)
            new_body = substitute(expr.body, expr.var, Var(new_var))
            return Lam(new_var, substitute(new_body, var, replacement))
        
        return Lam(expr.var, substitute(expr.body, var, replacement))
    
    elif isinstance(expr, App):
        return App(
            substitute(expr.func, var, replacement),
            substitute(expr.arg, var, replacement)
        )
    
    return expr

# Test
reset_fresh()
# (λx.x)[a/x] should stay λx.x
print(substitute(Lam('x', Var('x')), 'x', Var('a')))

# (λy.x)[a/x] should become λy.a
print(substitute(Lam('y', Var('x')), 'x', Var('a')))

# (λy.x)[y/x] should alpha-convert to avoid capture
print(substitute(Lam('y', Var('x')), 'x', Var('y')))

## Part 3: Beta Reduction

In [ ]:
def is_redex(expr: Expr) -> bool:
    """Check if expression is a redex (reducible expression)."""
    return isinstance(expr, App) and isinstance(expr.func, Lam)

def beta_reduce_step(expr: Expr) -> Tuple[Expr, bool]:
    """
    Perform one beta reduction step (leftmost-outermost).
    Returns (new_expr, did_reduce).
    """
    if isinstance(expr, Var):
        return expr, False
    
    elif isinstance(expr, Lam):
        new_body, reduced = beta_reduce_step(expr.body)
        if reduced:
            return Lam(expr.var, new_body), True
        return expr, False
    
    elif isinstance(expr, App):
        # Check if this is a redex
        if isinstance(expr.func, Lam):
            # Beta reduce!
            result = substitute(expr.func.body, expr.func.var, expr.arg)
            return result, True
        
        # Try reducing function first (leftmost)
        new_func, reduced = beta_reduce_step(expr.func)
        if reduced:
            return App(new_func, expr.arg), True
        
        # Try reducing argument
        new_arg, reduced = beta_reduce_step(expr.arg)
        if reduced:
            return App(expr.func, new_arg), True
        
        return expr, False
    
    return expr, False

# Test
reset_fresh()
expr = App(Lam('x', Var('x')), Var('a'))
print(f"Before: {expr}")
result, _ = beta_reduce_step(expr)
print(f"After: {result}")

In [ ]:
def evaluate(expr: Expr, max_steps: int = 100) -> Tuple[Expr, List[Expr], bool]:
    """
    Fully evaluate an expression.
    Returns (final_expr, steps, terminated).
    """
    steps = [expr]
    current = expr
    
    for _ in range(max_steps):
        new_expr, reduced = beta_reduce_step(current)
        if not reduced:
            return current, steps, True  # Reached normal form
        current = new_expr
        steps.append(current)
        
        # Detect obvious loops (same as previous)
        if len(steps) > 2 and str(steps[-1]) == str(steps[-2]):
            return current, steps, False  # Loop detected
    
    return current, steps, False  # Max steps reached

# Test: K a b → a
reset_fresh()
K = Lam('x', Lam('y', Var('x')))
expr = App(App(K, Var('a')), Var('b'))

result, steps, terminated = evaluate(expr)
print(f"Evaluating: {steps[0]}")
for i, step in enumerate(steps):
    print(f"  Step {i}: {step}")
print(f"Terminated: {terminated}")

## Part 4: Parser

In [ ]:
class Parser:
    """
    Parse lambda calculus expressions.
    
    Syntax:
      expr ::= var | 'λ' var '.' expr | '(' expr expr ')'
      var ::= letter+
    """
    
    def __init__(self, text: str):
        self.text = text.strip()
        self.pos = 0
    
    def parse(self) -> Expr:
        """Parse the entire expression."""
        expr = self.parse_expr()
        self.skip_whitespace()
        if self.pos < len(self.text):
            raise ValueError(f"Unexpected character at position {self.pos}: {self.text[self.pos]}")
        return expr
    
    def parse_expr(self) -> Expr:
        """Parse an expression."""
        self.skip_whitespace()
        
        if self.pos >= len(self.text):
            raise ValueError("Unexpected end of input")
        
        ch = self.text[self.pos]
        
        # Lambda abstraction
        if ch in 'λ\\':
            return self.parse_lambda()
        
        # Application (parenthesized)
        if ch == '(':
            return self.parse_application()
        
        # Variable
        if ch.isalpha():
            return self.parse_variable()
        
        raise ValueError(f"Unexpected character: {ch}")
    
    def parse_lambda(self) -> Lam:
        """Parse λx.body"""
        self.expect_one_of('λ\\')
        self.skip_whitespace()
        var = self.parse_var_name()
        self.skip_whitespace()
        self.expect('.')
        self.skip_whitespace()
        body = self.parse_expr()
        return Lam(var, body)
    
    def parse_application(self) -> App:
        """Parse (func arg)"""
        self.expect('(')
        self.skip_whitespace()
        func = self.parse_expr()
        self.skip_whitespace()
        arg = self.parse_expr()
        self.skip_whitespace()
        self.expect(')')
        return App(func, arg)
    
    def parse_variable(self) -> Var:
        """Parse a variable name."""
        name = self.parse_var_name()
        return Var(name)
    
    def parse_var_name(self) -> str:
        """Parse a variable name (letters only)."""
        start = self.pos
        while self.pos < len(self.text) and self.text[self.pos].isalpha():
            self.pos += 1
        if self.pos == start:
            raise ValueError(f"Expected variable name at position {self.pos}")
        return self.text[start:self.pos]
    
    def skip_whitespace(self):
        """Skip whitespace."""
        while self.pos < len(self.text) and self.text[self.pos].isspace():
            self.pos += 1
    
    def expect(self, ch: str):
        """Expect a specific character."""
        if self.pos >= len(self.text) or self.text[self.pos] != ch:
            raise ValueError(f"Expected '{ch}' at position {self.pos}")
        self.pos += 1
    
    def expect_one_of(self, chars: str):
        """Expect one of several characters."""
        if self.pos >= len(self.text) or self.text[self.pos] not in chars:
            raise ValueError(f"Expected one of '{chars}' at position {self.pos}")
        self.pos += 1

def parse(text: str) -> Expr:
    """Parse a lambda expression."""
    return Parser(text).parse()

# Test
print(parse("λx.x"))
print(parse("(λx.x a)"))
print(parse("λx.λy.x"))
print(parse("((λx.λy.x) a b)"))

## Part 5: REPL (Read-Eval-Print Loop)

In [ ]:
def repl_evaluate(text: str, max_steps: int = 50, show_steps: bool = True) -> None:
    """
    Parse and evaluate a lambda expression, showing the steps.
    """
    reset_fresh()
    
    try:
        expr = parse(text)
        print(f"Parsed: {expr}")
        print()
        
        result, steps, terminated = evaluate(expr, max_steps)
        
        if show_steps:
            print("Reduction trace:")
            for i, step in enumerate(steps):
                print(f"  {i}: {step}")
            print()
        
        if terminated:
            print(f"Result: {result}")
            print(f"Steps: {len(steps) - 1}")
        else:
            print(f"Did not terminate after {max_steps} steps")
            print(f"Last expression: {result}")
            
    except Exception as e:
        print(f"Error: {e}")

# Test cases
print("=" * 50)
print("Test 1: Identity")
repl_evaluate("(λx.x a)")

print("\n" + "=" * 50)
print("Test 2: K combinator")
repl_evaluate("((λx.λy.x) a b)")

print("\n" + "=" * 50)
print("Test 3: S combinator (partial)")
repl_evaluate("((λx.λy.λz.((x z) (y z))) a b)")

## Part 6: Validation Against Historical Data

In [ ]:
def validate_reduction(reduction_id: str) -> bool:
    """
    Validate our evaluator against a historical reduction trace.
    """
    trace = reductions[reductions['reduction_id'] == reduction_id].sort_values('step_number')
    
    if len(trace) == 0:
        print(f"No trace found for {reduction_id}")
        return False
    
    initial = trace.iloc[0]['initial_expression']
    expected_terminates = trace.iloc[-1]['terminates']
    expected_steps = trace.iloc[-1]['total_steps'] if expected_terminates else -1
    
    print(f"Validating: {reduction_id}")
    print(f"Expression: {initial}")
    print(f"Expected terminates: {expected_terminates}")
    print(f"Expected steps: {expected_steps}")
    
    try:
        reset_fresh()
        expr = parse(initial)
        result, steps, terminated = evaluate(expr, max_steps=100)
        
        print(f"Actual terminates: {terminated}")
        print(f"Actual steps: {len(steps) - 1}")
        
        if terminated == expected_terminates:
            if terminated and len(steps) - 1 == expected_steps:
                print("✓ PASSED")
                return True
            elif not terminated:
                print("✓ PASSED (both non-terminating)")
                return True
        
        print("✗ FAILED")
        return False
        
    except Exception as e:
        print(f"Error: {e}")
        return False

# Test some simple reductions
print("=" * 50)
validate_reduction('PR-001')

print("\n" + "=" * 50)
validate_reduction('PR-002')

In [ ]:
# Validate multiple reductions
def run_validation_suite(n: int = 10):
    """Run validation on the first n reduction traces."""
    ids = reductions['reduction_id'].unique()[:n]
    
    passed = 0
    failed = 0
    
    for rid in ids:
        print("=" * 50)
        if validate_reduction(rid):
            passed += 1
        else:
            failed += 1
        print()
    
    print("=" * 50)
    print(f"SUMMARY: {passed} passed, {failed} failed")

# Run on first 5 traces
run_validation_suite(5)

## Part 7: Church Encodings

In [ ]:
# Define Church encodings as parsed expressions
CHURCH = {
    'TRUE': parse("λx.λy.x"),
    'FALSE': parse("λx.λy.y"),
    'ZERO': parse("λf.λx.x"),
    'ONE': parse("λf.λx.(f x)"),
    'TWO': parse("λf.λx.(f (f x))"),
    'SUCC': parse("λn.λf.λx.(f ((n f) x))"),
    'PLUS': parse("λm.λn.λf.λx.((m f) ((n f) x))"),
    'MULT': parse("λm.λn.λf.(m (n f))"),
}

for name, expr in CHURCH.items():
    print(f"{name}: {expr}")

In [ ]:
# Build and evaluate an expression: SUCC ZERO
succ_zero = App(CHURCH['SUCC'], CHURCH['ZERO'])
print(f"SUCC ZERO = {succ_zero}")

reset_fresh()
result, steps, _ = evaluate(succ_zero, max_steps=20)
print(f"\nReduction:")
for i, step in enumerate(steps[:5]):  # First 5 steps
    print(f"  {i}: {step}")
print(f"  ...")
print(f"  Final: {result}")

## Part 8: Interactive Exploration

In [ ]:
def interactive_session():
    """
    Run an interactive REPL session.
    (In Colab, this won't work interactively, but shows the structure.)
    """
    print("Pure Passage Calculus Evaluator")
    print("Enter expressions or 'quit' to exit")
    print("Examples:")
    print("  (λx.x a)")
    print("  ((λx.λy.x) a b)")
    print()
    
    # Demo expressions
    demos = [
        "(λx.x a)",
        "((λx.λy.x) first second)",
        "((λx.λy.y) first second)",
        "(((λx.λy.λz.((x z) (y z))) (λa.a)) (λb.b))",
    ]
    
    for demo in demos:
        print(f"\n> {demo}")
        repl_evaluate(demo, show_steps=True)

interactive_session()

## Exercises

### Exercise 1: Extend the Parser

Modify the parser to support:
1. Multi-character variable names (already works!)
2. Alternative lambda syntax: `\x.body`
3. Let expressions: `let x = e1 in e2` → `(λx.e2) e1`

In [ ]:
# Exercise 1 workspace

# Test multi-character variables
print(parse("λfoo.λbar.(foo bar)"))

# Backslash already supported:
print(parse("\\x.x"))

### Exercise 2: Implement Eta Reduction

Add eta reduction to the evaluator:
`λx.(f x)` → `f` when `x` not free in `f`

In [ ]:
# Exercise 2 workspace

def is_eta_redex(expr: Expr) -> bool:
    """Check if expression is an eta redex: λx.(f x) where x not free in f."""
    # Your code here
    pass

def eta_reduce(expr: Expr) -> Expr:
    """Perform eta reduction if possible."""
    # Your code here
    pass

### Exercise 3: Non-Termination Detection

Improve non-termination detection:
1. Detect cycles longer than 1 step
2. Detect Omega-like patterns

In [ ]:
# Exercise 3 workspace

def detect_cycle(steps: List[Expr], window: int = 5) -> Optional[int]:
    """
    Detect if the last `window` steps form a cycle.
    Returns the cycle length if found, None otherwise.
    """
    # Your code here
    pass

## Summary

Congratulations! You've built a complete Passage Evaluator:

1. **AST**: Var, Lam, App data structures
2. **Free Variables**: Tracking unbound names
3. **Substitution**: Capture-avoiding replacement
4. **Beta Reduction**: The core computation rule
5. **Parser**: String to AST conversion
6. **Evaluator**: Multi-step reduction to normal form
7. **REPL**: Interactive exploration
8. **Validation**: Checking against historical data

You now have a working implementation of Kelleth Mund's Pure Passage Calculus—the formal system that emerged from the Great Categorical Collapse and founded the type theory tradition in Densworld.

---

## Course Complete!

You've completed **The Pure Passage Calculus**. Continue your journey with:

- **Course 2: Combinatorial Reduction** — Jorell Vance's name-free alternative
- **Course 3: Typed Passages** — Brennis Mund's solution to non-termination